TO DO:
- Implement differences between efficiency/power curve PSDs
- Update populated curve PSD to reflect Diameter units (Cell B3), Power (B4), Pwr/Eta cell (B5) 

##### Imports, File Setup

In [20]:
import os
import pandas as pd
import shutil
from openpyxl import load_workbook
import numpy as np
# import re

In [21]:
# This is the folder/file with the curve export csv
myDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\1 - PROJECTS\15 SP Integration\Curve PSD"
myFile = "PumpCurves - With Impeller Trims.csv"
filePath = os.path.join(myDir, myFile)

# This creates a dataframe of the curve export csv, and fills in the RPM(curve nominal) column
# data = pd.read_csv(filePath, sep=";", index_col=False, skip_blank_lines=False)
data = pd.read_csv(filePath, index_col=False, skip_blank_lines=False)
# data = data.replace(',','.', regex=True)
data.dropna(axis='index', how='all', inplace=True)
data['ProductNumber'] = data['ProductNumber'].ffill()

In [22]:
# This points to the curve PSD template to be used
templateDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\3 - RESOURCES\32 GXS"
template = "SKB Blank Curve PSD - Efficiency_Metric.xlsx"
template = os.path.join(templateDir, template)

# Create a local working copy to leave template unmodified
workingCopy = os.path.join(myDir, "SP Curve PSD - col A trim testing.xlsx")
shutil.copyfile(template, workingCopy)
wb = load_workbook(workingCopy)      

##### Functions

In [23]:
# def findSpeedCells(my_df):
#     """ function to find which cell to start populating curve data based on RPM"""
#     # from openpyxl.utils.cell import coordinate_from_string, column_index_from_string, get_column_letter
#     from openpyxl.utils.cell import get_column_letter
#     speedCells = []
    
#     max_cols = 21 * len(my_df.RPM.unique())
#     row = 7    
#     first_col = 4
#     diameter_cols = list(range(first_col,max_cols,21))
    
#     for item in diameter_cols:
#         col = get_column_letter(item)
#         cell_coordinate = "{}{}".format(col,row)
#         speedCells.append(cell_coordinate)      

#     return(speedCells)

In [24]:
# def add_a_curve(list_unique_curves):  # WORKING. DO NOT MODIFY YET
#     """ CREATE new tabs for each unique curve family """
    
#     global wb
#     tab_to_copy = wb['NEW'] 
    
#     partnumbers = []
#     min_speeds = []
#     nom_speeds = []
#     max_speeds = []
    
#     # Add a tab for each unique curve
#     for item in list_unique_curves: 
        
#         list_of_speeds = []

#         # Add 1 tab
#         tabName = str(item[0])
#         # print(tabName)
#         wb.copy_worksheet(tab_to_copy).title = tabName

#         # Fill in all speeds/trims in column A
#         for key, value in curveDataDict[tabName].iterrows():
#             list_of_speeds.append(value['RPM'])
        
#         speed_set = sorted(set(list_of_speeds), reverse=True)
#         # print(tabName, speed_set)  
        
#         for index, eachSpeed in enumerate(speed_set):
#             cell_name = "{}{}".format('A', 10+index)
#             curveSheet = wb[tabName]
#             curveSheet[cell_name].value = int(eachSpeed)

#             min_rpm, nom_rpm, max_rpm = get_min_max_values(speed_set)
            
#             partnumbers.append(tabName)
#             min_speeds.append(min_rpm)
#             nom_speeds.append(nom_rpm)
#             max_speeds.append(max_rpm)
            
#             zipped = list(zip(partnumbers, min_speeds, nom_speeds, max_speeds))
#             df = pd.DataFrame(zipped, columns=['Partnumber','Min RPM','Nominal RPM', 'Max RPM'])
        
#     return df
    
# # Adds curve tabs for each unique curve
# # minmax_speeds_df = add_a_curve(uniqueCurvePartnumbers)

In [25]:
def curveDataMatches(curveDataDict):  
    """ Create list of PNs that share curve data """

    curveDataCopy = curveDataDict.copy()
    listOfLists = []
    
    for eachPN, eachDF in curveDataDict.items():  
        listOfPNs = []
        for key, value in curveDataCopy.items():
            if value.equals(eachDF):
                listOfPNs.append(key)
        
        if listOfPNs not in listOfLists:
            listOfLists.append(listOfPNs)
    
    # print(f'list of lists: {listOfLists}')
    # print(f'list of pns: {listOfPNs}')
    
    return listOfLists

##### Testing

##### Main

In [26]:
# Creates dictionary with part numbers as keys, curves as dataframes for each key
grouped = data.groupby(data.ProductNumber)

dict_curveData = {}
for productNumber, sub_df in grouped:
    dict_curveData.update({productNumber:sub_df})

In [27]:
# Creates list of which part numbers share curve data
uniqueCurvePartnumbers = curveDataMatches(dict_curveData)

In [28]:
# Add 1 curve tab to workbook for every unique curve found (avoids duplicating data in SKB)
for item in uniqueCurvePartnumbers: 
    tabName = str(item[0])
    wb.copy_worksheet(wb['NEW']).title = tabName # Creates and renames blank PSD Tab as template for each new curve tab

In [29]:
def extract_variable_speeds(df) -> list:
    ''' Takes curve data in dataframe format, and returns list of all unique speeds for variable speed pumps'''
    list_of_rpms = []

    for key, value in df.iterrows():
        rpm_value = value['RPM(Curve nominal)']
        if not np.isnan(rpm_value):
            list_of_rpms.append(rpm_value)
    
    list_of_rpms = sorted(set(list_of_rpms), reverse=True)

    return list_of_rpms

In [30]:
def extract_variable_trims(df) -> list:
    ''' Takes curve data in dataframe format, and returns list of all trims for trimmable pumps'''
    list_of_trims = []

    for key, value in df.iterrows():
        trim_value = value['Impeller size']
        if not np.isnan(trim_value):
            list_of_trims.append(trim_value)
    
    list_of_trims = sorted(set(list_of_trims), reverse=True)

    return list_of_trims

In [ ]:
# Iterate through list of unique curve names, and populate the corresponding workbook tab
for uniqueCurve in uniqueCurvePartnumbers:
    curveSheet = wb[uniqueCurve[0]]
    curve_data_df = dict_curveData[uniqueCurve[0]]
    
    # Fill Speed or trim in column A
    # speeds = extract_variable_speeds(curve_data_df)
    trims = extract_variable_trims(curve_data_df)

    for index, eachSpeedTrim in enumerate(trims):
        cell_name = "{}{}".format('A', 10+index)
        curveSheet[cell_name].value = int(eachSpeedTrim)
        # print(f'For PN: {uniqueCurve[0]} -> cell: {cell_name} should have value: {int(eachSpeed)}. It currently has: {curveSheet[cell_name].value}')

    # Fill Curve, power/eff, NPSH data
    cellName = 'D7' 
    first_row_offset = 3

    for key, value in curve_data_df.iterrows():
        print(value)
        curveSheet[cellName].offset(first_row_offset + key, 0).value = value['Q']
        curveSheet[cellName].offset(first_row_offset + key, 1).value = value['H']
        curveSheet[cellName].offset(first_row_offset + key, 7).value = value['Q']
        # curveSheet[cellName].offset(first_row_offset + key, 8).value = value['Eta1']
        curveSheet[cellName].offset(first_row_offset + key, 8).value = value['P1']
        curveSheet[cellName].offset(first_row_offset + key,14).value = value['Q']
        curveSheet[cellName].offset(first_row_offset + key,15).value = value['NPSH']
    

In [234]:
# def speeds_for_curve_header_data():
    # # partnumbers = []
    # # min_speeds = []
    # # nom_speeds = []
    # # max_speeds = []

    # min_rpm, nom_rpm, max_rpm = get_min_max_values(speed_set)

    # partnumbers.append(tabName)
    # min_speeds.append(min_rpm)
    # nom_speeds.append(nom_rpm)
    # max_speeds.append(max_rpm)

    # zipped = list(zip(partnumbers, min_speeds, nom_speeds, max_speeds))
    # df = pd.DataFrame(zipped, columns=['Partnumber','Min RPM','Nominal RPM', 'Max RPM'])
#     return

In [235]:
# # Fill out Curve Header Data Tab
# newPumpFamilyName = "SP"
# sheet = wb["Curve Header Data"]
# sheet['B7'] = newPumpFamilyName

# cellName = 'A1' 
# first_row_offset = 10

# for index, speedData in df_of_speeds.iterrows():
#     sheet[cellName].offset((first_row_offset + index), 0).value = speedData['Partnumber']
#     sheet[cellName].offset((first_row_offset + index), 2).value = speedData['Nominal RPM']
#     sheet[cellName].offset((first_row_offset + index), 6).value = speedData['Nominal RPM']
#     sheet[cellName].offset((first_row_offset + index), 7).value = speedData['Min RPM']
#     sheet[cellName].offset((first_row_offset + index), 8).value = speedData['Max RPM']

In [236]:
def fillCurveHeaderDataTab(df_of_speeds):
    global wb
    newPumpFamilyName = "SP"
    sheet = wb["Curve Header Data"]

    cellName = 'A1' 
    first_row_offset = 10

    for index, speedData in df_of_speeds.iterrows():
        sheet[cellName].offset((first_row_offset + index), 0).value = speedData['Partnumber']
        sheet[cellName].offset((first_row_offset + index), 2).value = speedData['Nominal RPM']
        sheet[cellName].offset((first_row_offset + index), 6).value = speedData['Nominal RPM']
        sheet[cellName].offset((first_row_offset + index), 7).value = speedData['Min RPM']
        sheet[cellName].offset((first_row_offset + index), 8).value = speedData['Max RPM']

    sheet['B7'] = newPumpFamilyName
    wb.save(workingCopy)

    return

# fillCurveHeaderDataTab(minmax_speeds_df)

In [237]:
# Create list of PNs that share curve data, and inserts tab to illustrate
curveFamilySheet = wb.create_sheet("Shared Curves", 2)

for index, family in enumerate(uniqueCurvePartnumbers):
    row = index + 1  # Excel doesn't like 0-indexes

    # Fill 1st column in spreadsheet
    curveFamilySheet.cell(row=row, column=1).value = "Curve " + str(row)

    # Fill 2nd (or more) columns with partnumbers that share curves
    if len(family) == 1:
        curveFamilySheet.cell(row=row, column=2).value = family[0]
        # print('length of family is 1')
    else:
        curveFamilySheet.cell(row=row, column=2).value = family[0]
        curveFamilySheet.cell(row=row, column=3).value = family[1]
        # print(f'family[0]: {family[0]}, family[1]: {family[1]}')

In [32]:
# Save changes to excel sheet
wb.save(workingCopy)